In [58]:
import numpy as np
import scipy
import utils
from utils import HouseHolder, QR, SVD

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Problem 1: SVD by Two-Phase Approach

## Phase-I: Golub-Kahan Bidiagonalization

In [59]:
%precision 8

'%.8f'

In [60]:
A = np.array([[0, 0, 0, 0],
              [0, 0, 0, 0],
              [0, 0, 1, 0],
              [0, 0, 0, 0],
              [2, 5, 0, 0],
              [0, 0, 0, 0],
              [0, 0, 0, 0]], dtype=np.float64)


# A = np.array([[1, 0, 1],
#               [2, 5**.5, 0],
#               [0, 0, 1],
#               [0, 0, 1]])
B, Qt, P = SVD.svd_phaseI(A)
print(B)


[[ 2. -5.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0. -1.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]]


In [61]:
print(Qt @ A @ P)

[[ 2. -5.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0. -1.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]]


Todo: We may not need to form the reflection matrix H explicitly during Golub-Kahan bidiagonalization.

## Phase-II-A

**Test SVD**

In [62]:
%precision 4

'%.4f'

In [81]:
m=  n = 1024
A = np.random.rand(m,n)


In [64]:
X = np.array([[0, 0, 0, 0],
              [0, 0, 0, 0],
              [0, 0, 1, 0],
              [0, 0, 0, 0],
              [2, 5, 0, 0],
              [0, 0, 0, 0],
              [0, 0, 0, 0]], dtype=np.float64)

In [82]:
U, S, Vt = SVD.svd(A)
# print(U@np.diag(S)@Vt)
print(np.sum(np.abs(U@np.diag(S)@Vt - A)< 1e-13))
U, S, Vt

phaseI: 42.835978746414185
phaseII: 71.46458125114441
1048576


(array([[-0.0312,  0.0019,  0.0277, ..., -0.0236,  0.0141,  0.0423],
        [-0.0312,  0.0245, -0.0304, ...,  0.0071,  0.0093, -0.0416],
        [-0.0309, -0.0137,  0.0553, ..., -0.0084,  0.025 ,  0.0134],
        ...,
        [-0.0311, -0.0544,  0.0087, ...,  0.0189, -0.0073,  0.0122],
        [-0.0311,  0.0012,  0.0645, ..., -0.0265,  0.0306,  0.0011],
        [-0.0312, -0.0382, -0.03  , ..., -0.0062, -0.0239, -0.0105]]),
 array([5.1210e+02, 1.8303e+01, 1.8258e+01, ..., 2.9334e-02, 1.5236e-02,
        3.3635e-03]),
 array([[-3.1178e-02, -3.0464e-02, -3.0865e-02, ..., -3.0801e-02,
         -3.0778e-02, -3.0611e-02],
        [ 3.1722e-02, -6.7891e-02,  5.6502e-02, ..., -6.2018e-05,
          3.8338e-02, -4.0348e-03],
        [-1.7848e-02, -1.3250e-02, -1.8052e-02, ..., -8.5527e-03,
         -5.2152e-02,  3.1447e-02],
        ...,
        [ 2.5326e-02,  4.3156e-03, -4.4283e-02, ..., -2.3825e-02,
          3.6184e-02, -4.8622e-02],
        [ 4.6897e-02,  1.8263e-02, -2.5546e-02, ..., -1

In [75]:
U, S, Vt  = scipy.linalg.svd(A, full_matrices=False)
# print(U@np.diag(S)@Vt)
U, S, Vt

(array([[-0.0318,  0.024 , -0.0258, ...,  0.0072,  0.0386, -0.0329],
        [-0.0317,  0.0404,  0.0006, ..., -0.0211, -0.0075,  0.0624],
        [-0.0315,  0.0322, -0.0414, ..., -0.0081,  0.0072,  0.0065],
        ...,
        [-0.031 , -0.0359,  0.0533, ..., -0.0156, -0.0283, -0.0327],
        [-0.0321,  0.0266, -0.0513, ..., -0.0212, -0.0254, -0.0007],
        [-0.0319,  0.0087, -0.0075, ...,  0.0284,  0.0568, -0.0342]]),
 array([5.1281e+02, 1.8356e+01, 1.8266e+01, ..., 2.7355e-02, 1.5270e-02,
        8.7526e-03]),
 array([[-0.0312, -0.031 , -0.0303, ..., -0.0312, -0.0319, -0.0313],
        [ 0.0612,  0.0031,  0.0503, ..., -0.0586,  0.0193,  0.0104],
        [-0.0133, -0.0307,  0.0154, ...,  0.0075, -0.0673, -0.0399],
        ...,
        [ 0.0075,  0.0456,  0.0185, ...,  0.03  , -0.028 , -0.0091],
        [ 0.0511,  0.0295, -0.0266, ...,  0.051 , -0.0097, -0.0299],
        [ 0.0454,  0.0333, -0.0338, ...,  0.0124, -0.0061,  0.0035]]))